---

# Assignment 3

```
X1 ------\
          \
          (P) ---- (P) ---- (T)
          /
X2 ------/
         
```

1. Create a two layer neural network with one perceptron in each layer (see Diagram above). Write a validation code that does along with your implementation. The goal of network is to optimize the two perceptrons to produce the output target `T` given the inputs `X1` and `X2`. Assume the output `O` of each perceptron is

$$ O = \sigma{(w1*x1 + w2*x2 + b)} $$ where

$$\sigma(x) = \frac{1}{1+e^{-x}}$$

Feel free to change the loss function if you like.



In [1]:
import numpy as np

In [2]:
class Node(object):
    """
    Base object for all inputs and outputs.
    """
    def __init__(self, value, grad):
        self.value = value
        self.gradient = grad

In [3]:
class MultiplyNode(object):
    """
    Multiplies two inputs
    """
    def forward(self, *args):
        self.inputs = []
        
        for i in args:
            self.inputs.append(i)
        
        self.output = Node(self.inputs[0].value * self.inputs[1].value, 0)
        return self.output
    
    def backward(self):
        self.inputs[0].gradient = self.inputs[1].value * self.output.gradient
        self.inputs[1].gradient = self.inputs[0].value * self.output.gradient

In [4]:
class AddNode(object):    
    """
    Adds two inputs x1 and x2.
    """
    def forward(self, nodes):
        self.inputs = nodes
             
        self.output = Node(self.inputs[0].value + self.inputs[1].value, 0)
        return self.output
    
    def backward(self):
        self.inputs[0].gradient = 1 * self.output.gradient
        self.inputs[1].gradient = 1 * self.output.gradient

In [5]:
class SigmoidNode(object):
    """
    Adds a sigmoid non-linearity to a single input
    """
    def forward(self, x):
        self.x = x
        self.output = Node(1/(1 + np.exp(-1 * self.x.value)), 0.0)
        return self.output
        
    def backward(self):
        s = 1/(1 + np.exp(-1 * self.x.value))
        self.x.gradient = (s * (1 - s)) * self.output.gradient

In [6]:
class Perceptron(object):
    ### Consider first two inputs 0 if not provided
    def __init__(self, alpha =0.01,x1=0,x2=0, **kwargs):
        ### Hyper parameters
        self.alpha = alpha
        
        ### Set inputs
        self.inputs = [Node(x1,0.0),Node(x2,0.0)]
        for i in kwargs:
            input = Node(kwargs[i],0.0)
            self.inputs.append(input)
        
        ### Initializing weights to a random float between -1 and 1
        self.weights = []
        for i in self.inputs:
            weight = Node(np.random.uniform(-1,1),0.0)
            self.weights.append(weight) 
        
        ### Initialize bias (just one for now) to a random float between -1 and 1
        self.bias = Node(np.random.uniform(-1,1),0.0)
        
        ### Initialize Operators
        self.initialize_operators()
        
    def initialize_operators(self):
        ### Multiply inputs with weights operator
        self.weighted_input_mult_op =[]
        for i in range(len(self.inputs)):
            self.weighted_input_mult_op.append(MultiplyNode())
            
        ### Add all weighted inputs operator
        self.sum_of_all_weighted_inputs_op = AddNode()
        
        ### Add bias to the sum of all weighted inputs operator
        self.sum_of_weighted_inputs_and_bias_op = AddNode()
        
        ### Sigmoid
        self.sigmoid = SigmoidNode()
        
        
    def forward(self):
        ### Multiply inputs with weights
        self.weighted_inputs = []
        for i in range(len(self.weighted_input_mult_op)):            
            input_mul_weight = self.weighted_input_mult_op[i].forward(self.inputs[i],self.weights[i])
            self.weighted_inputs.append(input_mul_weight)
               
        
        ### Add all weighted inputs
        self.sum_of_all_weighted_inputs = self.sum_of_all_weighted_inputs_op.forward(self.weighted_inputs)
            
        
        ### Add bias to the sum of all weighted inputs
        self.sum_of_weighted_inputs_and_bias = self.sum_of_weighted_inputs_and_bias_op.forward([self.sum_of_all_weighted_inputs,self.bias])
        
        self.sigmoid.forward(self.sum_of_weighted_inputs_and_bias)
        
    
    def backward(self):
        self.sigmoid.backward()
        self.sum_of_all_weighted_inputs_op.backward()
        self.sum_of_weighted_inputs_and_bias_op.backward()
        for i in range(len(self.weighted_inputs)):
            self.weighted_input_mult_op[i].backward()
            
    def update(self):
        for i in range(len(self.weights)):
            self.weights[i].value -= self.alpha * self.weights[i].gradient
            
        self.bias.value -= self.alpha * self.bias.gradient

In [7]:
p0 = Perceptron(alpha = 0.2, x1 =3,x2=5)
p1 = Perceptron(alpha = 0.2)
# number of iterations
N = 100000
# expected output 
target = 0.784

for i in range(N):
    p0.forward()
   
    p1.inputs.append(Node(p0.sigmoid.output.value,0.0))
    p1.forward()
    p1.sigmoid.output.gradient = -2 * (target - p1.sigmoid.output.value)
    p1.backward()
    p1.update()

    p0.sigmoid.output.gradient = -2 * (target - p0.sigmoid.output.value)
    p0.backward()
    p0.update()

p1.sigmoid.output.value

0.7839999999999985

In [8]:
p0 = Perceptron(alpha = 0.2, x1 =3,x2=5)
p1 = Perceptron(alpha = 0.2, x1=8,x2=1,x3=9)
p2 = Perceptron(alpha = 0.2)
# number of iterations
N = 100000
# expected output 
target = 0.235

for i in range(N):
    p0.forward()
    
    p1.forward()
    p2.inputs.append(Node(p0.sigmoid.output.value,0.0))
    p2.inputs.append(Node(p1.sigmoid.output.value,0.0))
    p2.forward()
    p2.sigmoid.output.gradient = -2 * (target - p2.sigmoid.output.value)
    
    p2.backward()
    p2.update()

    p0.sigmoid.output.gradient = -2 * (target - p0.sigmoid.output.value)
    p1.sigmoid.output.gradient = -2 * (target - p1.sigmoid.output.value)
    p0.backward()
    p1.backward()
    p0.update()
    p1.update()

p2.sigmoid.output.value

0.2350000000000015